In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [10]:
client = pd.read_csv("data_tratada/client.csv", sep=",", na_values=["?"])
client.head()


,client_id,district_id,sex,birth_date
0,1,18,f,1970-12-13
1,2,1,m,1945-02-04
2,3,1,f,1940-10-09
3,4,5,m,1956-12-01
4,5,5,f,1960-07-03


In [4]:
collected_date = pd.to_datetime("2000-01-01", format="%Y-%m-%d")


In [5]:
def getAge(row):
    date = pd.to_datetime(row['birth_date'], format='%Y-%m-%d')
    return collected_date.year - date.year - ((collected_date.month, collected_date.day) < (date.month, date.day))

In [11]:
client['age'] = client.apply(lambda x : getAge(x), axis=1)


In [7]:
client = client.drop(client.columns[[3]], axis=1)
client.head()

,client_id,district_id,sex,age
0,1,18,f,29
1,2,1,m,54
2,3,1,f,59
3,4,5,m,43
4,5,5,f,39


In [9]:
demographics = pd.read_csv("data_tratada/demographics.csv", sep=",", header=0)
demographics = demographics.rename(columns={"code" : "district_id"})
client_demographics = pd.merge(client, demographics, on="district_id")
client_demographics.head()


,client_id,district_id,sex,age,no. of inhabitants,ratio of urban inhabitants,average salary,crime_increase,unemployment_increase,crime_avg,unemployment_avg
0,1,18,f,29,70699,65.3,8968,2.4,0.52,25.81,3.09
1,420,18,m,21,70699,65.3,8968,2.4,0.52,25.81,3.09
2,499,18,f,64,70699,65.3,8968,2.4,0.52,25.81,3.09
3,519,18,m,19,70699,65.3,8968,2.4,0.52,25.81,3.09
4,682,18,m,20,70699,65.3,8968,2.4,0.52,25.81,3.09


In [11]:
client_demographics = client_demographics.sort_values(by=['client_id'])
client_demographics.to_csv("data_tratada/client_demographics.csv", index=False, float_format="%.2f")


### Associantion between client_demo and Account ID's

In [38]:
client_acount_association = pd.read_csv("data/disp.csv", sep=";", header=0)
client_acount_association.head()


,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


Merge the two tables by client_id

In [49]:
merged_client_demo_account = pd.merge(client_acount_association, client_demographics, on= 'client_id')
merged_client_demo_account.to_csv("data_tratada/client_demo_account.csv", index=False, float_format="%.2f")
merged_client_demo_account.head()



,disp_id,client_id,account_id,type,district_id,sex,age,no. of inhabitants,ratio of urban inhabitants,average salary,crime_increase,unemployment_increase,crime_avg,unemployment_avg
0,1,1,1,OWNER,18,f,29,70699,65.3,8968,2.40,0.52,25.81,3.09
1,2,2,2,OWNER,1,m,54,1204953,100.0,12541,11.15,0.14,76.68,0.36
2,3,3,2,DISPONENT,1,f,59,1204953,100.0,12541,11.15,0.14,76.68,0.36
3,4,4,3,OWNER,5,m,43,95616,51.4,9307,4.43,0.58,29.58,4.14
4,5,5,3,DISPONENT,5,f,39,95616,51.4,9307,4.43,0.58,29.58,4.14


### Merge the Loan table and the table with all the client Info


In [46]:
loan = pd.read_csv("data/loan_dev.csv", sep=";", header=0)
loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1


In [52]:
predeiction_table = pd.merge(merged_client_demo_account, loan, on= 'account_id')
predeiction_table.to_csv("data_tratada/prediction_table.csv", index=False, float_format="%.2f")
predeiction_table.head()


,disp_id,client_id,account_id,type,district_id,sex,age,no. of inhabitants,ratio of urban inhabitants,average salary,crime_increase,unemployment_increase,crime_avg,unemployment_avg,loan_id,date,amount,duration,payments,status
0,2,2,2,OWNER,1,m,54,1204953,100.0,12541,11.15,0.14,76.68,0.36,4959,940105,80952,24,3373,1
1,3,3,2,DISPONENT,1,f,59,1204953,100.0,12541,11.15,0.14,76.68,0.36,4959,940105,80952,24,3373,1
2,25,25,19,OWNER,21,f,60,103347,67.0,9104,0.53,0.56,22.51,1.79,4961,960429,30276,12,2523,-1
3,78,78,67,OWNER,16,f,55,93931,56.9,8427,0.42,0.42,20.16,1.33,4973,960502,165960,24,6915,1
4,158,158,132,OWNER,40,f,54,128118,85.3,9317,-0.60,0.58,53.94,6.78,4996,961106,88440,12,7370,1
